In [ ]:
from __future__ import print_function
from pyspark.sql.types import StringType ,FloatType
from pyspark.sql.functions import udf
#import org.apache.spark.sql.functions.monotonicallyIncreasingId
from pyspark import SparkContext
from pyspark import SparkConf, SparkContext ,SQLContext,Row,HiveContext
import pyspark.sql.functions as func
from pyspark.sql.functions import lit
from datetime import datetime
from pyspark.sql.functions import col
from functools import reduce  
from pyspark.sql import DataFrame
#import org.apache.spark.sql.expressions._
from pyspark import HiveContext
from pyspark.sql.types import *
from pyspark.sql import Row, functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import broadcast
from datetime import datetime
from datetime import timedelta
import hashlib
import sys
import random
from random import randint

#sc = SparkContext(appName="Data_creation_process")
#sqlContext=HiveContext(sc)

sqlContext.setConf("hive.exec.dynamic.partition ", "true")
sqlContext.setConf("hive.exec.dynamic.partition.mode", "true")
sqlContext.setConf("hive.execution.engine", "spark")
sqlContext.setConf("hive.vectorized.execution.enabled", "true")
sqlContext.setConf("hive.vectorized.execution.reduce.enabled", "true")

load_dt="2017-06-15"
cols_select = ['prod_price',
               'prod_feat_1',
               'prod_feat_2',
               'cust_age',
               'prod_feat_3',
               'cust_region',
               'prod_type',
               'cust_sex',
               'cust_title',
               'feedback']


### 3 fileds are used for this account table.

def build_schema():
    print("Build and return a schema to use for the sample data.")
    schema = StructType(
        [
            StructField("prod_price",  StringType(),  True),
            StructField("prod_feat_1", StringType(),  True),
            StructField("prod_feat_2", StringType(),  True),
            StructField("cust_age",    StringType(),  True),
            StructField("prod_feat_3", StringType(),  True),
            StructField("cust_region", StringType(),  True),
            StructField("prod_type",   StringType(),  True),
            StructField("cust_sex",    StringType(),  True),
            StructField("cust_title",  StringType(),  True),
            StructField("feedback",    StringType(),  True)
        ]
    )
    return schema


rec_cnt=5000
def build_data(cnt):
    rec = []
    for i in range(1,rec_cnt):
        feedback = random.choice(["Negative" ,"Positive","None","Neutral"])
        cust_title = random.choice(["Director","Manager","Developer","Nurse","Doctor","BarTender",""])
        cust_sex = random.choice(["M","F"])
        prod_type = random.choice(["Home","Cleaning"])
        cust_region = random.choice(["North","South","East","West","Central"])
        prod_feat_3 = random.choice(["A","C","D","E"])
        cust_age = str(randint(10,70))
        prod_feat_2 = str(random.uniform(1.5,2.6))
        prod_feat_1 = str(random.uniform(11.5, 22.6))
        prod_price = str(random.uniform(111.5, 2322.6))

        rec.append(prod_price + "~" +
                   prod_feat_1 + "~" +
                   prod_feat_2 + "~" +
                   cust_age + "~" +
                   prod_feat_3 + "~" +
                   cust_region + "~" +
                   prod_type + "~" +
                   cust_sex + "~" +
                   cust_title + "~" +
                   feedback  )
    return (rec)


hist_data = build_data(int(rec_cnt))
dt = datetime.now()
load_id = ('%02d%02d%02d%02d%02d%02d%d' % (
dt.year, dt.month, dt.day, dt.hour, dt.minute, dt.second, dt.microsecond))[:-4]
# load_dt=dt.strftime('%Y%m%d')
load_tm = dt.strftime('%Y-%m-%d %H:%M:%S')
eff_start_date = dt.strftime('%Y-%m-%d')
histrawInput = sc.parallelize(hist_data)
histrawInputSplit = histrawInput.map(lambda x: x.split("~"))
hist_df = sqlContext.createDataFrame(histrawInputSplit, schema=build_schema())
hist_process_df = hist_df.select(hist_df.prod_price.cast('float'), \
                  hist_df.prod_feat_1.cast('float'), \
                  hist_df.prod_feat_2.cast('float'), \
                  hist_df.cust_age.cast('int'), \
                  *cols_select[4:]).withColumn("load_tm", lit(load_tm)).withColumn("load_date", lit(load_dt))
hist_process_df.write.mode("append").partitionBy('load_date').saveAsTable('cust_rf.cust_prod_detl')


Build and return a schema to use for the sample data.
